In [11]:
import os
from glob import glob

import pandas as pd

In [ ]:


def reshape_clause_file(file_path):
    """
    Reshape an Excel file containing legal clauses from wide to long format.
    
    Parameters
    ----------
    file_path : str
        Path to the Excel file containing clause data.

    Returns
    -------
    pandas.DataFrame
        A dataframe in long format with columns: filename, clause_type, and clause_text.
        Returns an empty dataframe with expected columns if processing fails.
    """
    try:
        df = pd.read_excel(file_path)

        # Ensure first column is 'filename' and others are clause types
        df.columns = df.columns.astype(str)  # Ensure all columns are string
        filename_col = df.columns[0]

        # Check for duplicate column names
        if df.columns.duplicated().any():
            raise ValueError(f"Duplicate columns found in: {file_path}")

        # Safely melt to long format
        df_long = df.melt(id_vars=[filename_col], var_name="clause_type", value_name="clause_text")
        df_long = df_long.dropna(subset=["clause_text"])  # Drop empty clause text rows

        # Rename columns
        df_long = df_long.rename(columns={filename_col: "filename"})

        # Reset index to avoid index-related errors in concat
        return df_long.reset_index(drop=True)

    except Exception as e:
        print(f"⚠️ Error processing {file_path}: {e}")
        return pd.DataFrame(columns=["clause_type", "clause_text", "filename"])

In [19]:
# Define your data folder
directory = "Datasets/CUAD/CUAD clauses"

# Get only real Excel files (ignore temporary Excel files starting with ~$)
excel_files = [
    f for f in glob(os.path.join(directory, "*.xlsx")) if not os.path.basename(f).startswith("~$")
]

# Process and collect all clauses
reshaped_list = []
for file in excel_files:
    try:
        reshaped_list.append(reshape_clause_file(file))
    except Exception as e:
        print(f"⚠️ Error processing {file}: {e}")

# Combine all successfully reshaped data
all_clauses = pd.concat(reshaped_list, ignore_index=True)

# Preview or save
print(all_clauses.head(5))
all_clauses.to_csv("all_reshaped_clauses.csv", index=False)

                                            filename        clause_type  \
0  GopageCorp_20140221_10-K_EX-10.1_8432966_EX-10...  Change of Control   
1  DeltathreeInc_19991102_S-1A_EX-10.19_6227850_E...  Change of Control   
2  EdietsComInc_20001030_10QSB_EX-10.4_2606646_EX...  Change of Control   
3  MusclepharmCorp_20170208_10-KA_EX-10.38_989358...  Change of Control   
4  TomOnlineInc_20060501_20-F_EX-4.46_749700_EX-4...  Change of Control   

                                         clause_text  
0  For purposes of the preceding sentence, and wi...  
1  The term of this Agreement shall be effective ...  
2  For purposes of this Agreement, "Change in Con...  
3  Neither party shall voluntarily or by operatio...  
4  Notwithstanding the foregoing, Skype or Skype ...  
